In [2]:
from pydantic import BaseModel
class UserResponse(BaseModel):
    id: int
    name: str

user = UserResponse(id=1, name="y.morimoto")
vars(user)

{'id': 1, 'name': 'y.morimoto'}

In [ ]:

url = "/documents/dfge5ttedfgd"


ParseResult(scheme='', netloc='', path='/documents/dfge5ttedfgd', params='', query='', fragment='')

In [70]:
import re
path_pattern = '/documents/{document_id}/{id}'.split('/')
request_path = '/documents/sidf0930-s9d90/aaa-bbb'.split('/')
params_dict = {}
for i in range(len(path_pattern)):
    m = re.match(r"\{[a-z0-9_].*?\}", path_pattern[i])
    if m:
        path_param = re.sub(r"[\{,\}]", '', path_pattern[i])
        params_dict[path_param] = request_path[i]

params_dict

{'document_id': 'sidf0930-s9d90', 'id': 'aaa-bbb'}

In [77]:
import re
path = "/documents/{id}/{document_id}"
re.sub(r"\{[a-z0-9_].*?\}", "{#}", path)

'/documents/{#}/{#}'

In [120]:
import re

class Router():
    def __init__(self):
        self.decorated_functions = {}

    def get(self, path: str):
        def inner(func):
            key = self.gen_func_key('get', path)
            self.decorated_functions[key] = {"path": path, "function": func}
        return inner

    def post(self, path: str):
        def inner(func):
            key = self.gen_func_key('post', path)
            self.decorated_functions[key] = {"path": path, "function": func}
        return inner

    def gen_func_key(self, method: str, path: str):
        # example result: 'get-/documents/{#}/{#}'
        path_segments = path.strip('/').split('/')
        root_segment = path_segments[0]
        segments = "/".join(['{#}']*len(path_segments[1:]))
        return method.lower() + "-" + f"/{root_segment}/{segments}"

    def get_path_params(self, path_pattern: str, request_path: str):
        params_dict = {}
        path_pattern = path_pattern.split('/')
        request_path = request_path.split('/')
        for i in range(len(path_pattern)):
            m = re.match(r"\{[a-z0-9_].*?\}", path_pattern[i])
            if m:
                path_param = re.sub(r"[\{,\}]", '', path_pattern[i])
                params_dict[path_param] = request_path[i]

        return params_dict

    def handler(self, event):
        method = event["httpMethod"]
        path = event["path"]
        key = self.gen_func_key(method, path)
        target = self.decorated_functions[key]
        kwargs = self.get_path_params(target['path'], path)
        target['function'](**kwargs)

router = Router()

@router.get('/documents/{id}/{document_id}/chunk')
def get_documents(id: str, document_id: str):
    print("ドキュメントの取得: id=", id)
    print("ドキュメントの取得: document_id=", document_id)

@router.post('/documents/{id}/{document_id}/chunk')
def update_documents(id: str, document_id: str):
    print("ドキュメントの更新: id=", id)
    print("ドキュメントの更新: document_id=", document_id)

print(router.decorated_functions)

# --- lambda handler ---
payload = {"httpMethod": "GET", "path": "/documents/sidf0930-s9d90/sd89sd0-sasd3a/chunk"}
router.handler(payload)

{'get-/documents/{#}/{#}/{#}': {'path': '/documents/{id}/{document_id}/chunk', 'function': <function get_documents at 0x1106e23e0>}, 'post-/documents/{#}/{#}/{#}': {'path': '/documents/{id}/{document_id}/chunk', 'function': <function update_documents at 0x1106e22a0>}}
ドキュメントの取得: id= sidf0930-s9d90
ドキュメントの取得: document_id= sd89sd0-sasd3a


In [97]:
method = "get"
path = "/documents/sidf0930-s9d90/asdfsdf"
path_segments = path.strip('/').split('/')
root_segment = path_segments[0]
segments = "/".join(['{#}']*len(path_segments[1:]))
method + "-" + f"{root_segment}/{segments}"

'get-documents/{#}/{#}'